In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from functions import make_point, up_load_gdb, up_load_shp
from forecast import importForcast

In [2]:
forecast=importForcast()
forecast

,Taz_num,Name_hebre,main_sector,classification_name,Shape_Length,Shape_Area,Student_by_Classrooms,geometry
0,3008.0,חניון נצבא,Jewish,כללי צעיר,1529.406023,127367.285166,0,"MULTIPOLYGON Z (((220343.956 627834.158 0.000,..."
1,3312.0,סטריפ דרך חברון דרום,Jewish,כללי צעיר,2056.390116,129610.690438,1,"MULTIPOLYGON Z (((220257.628 627500.897 0.000,..."
2,3311.0,גבעת המטוס,Jewish,כללי צעיר,2263.979987,231733.517211,0,"MULTIPOLYGON Z (((220082.056 627554.698 0.000,..."
3,3601.0,בית צפפא מזרח,Arab,ערבי מאוזן,1489.869222,150074.543659,0,"MULTIPOLYGON Z (((220100.950 627593.769 0.000,..."
4,3009.0,סטריפ דרך חברון צפון,Jewish,כללי צעיר,1071.062705,67863.168750,1,"MULTIPOLYGON Z (((220414.404 627803.685 0.000,..."
5,3007.0,דרך חברון אשר וינר,Jewish,כללי צעיר,1560.865752,59570.861254,0,"MULTIPOLYGON Z (((220343.956 627834.158 0.000,..."


In [3]:
def add_geographical_Features(forecast):
    forecast_point = make_point(forecast)

    subdistrict_il=up_load_gdb( r'C:\Users\dpere\Documents\JTMT\creat_forecast_ad_hoc\data\needed_files\subdistrict2008.gdb','subdistrict2008_ITM')

    muni_JTMT=up_load_gdb( r'C:\Users\dpere\Documents\JTMT\creat_forecast_ad_hoc\data\needed_files\MUNI_border.gdb','muni_under_JTMT_ITM')

    jeru_metro_jtmt_border=up_load_shp( r'C:\Users\dpere\Documents\JTMT\creat_forecast_ad_hoc\data\needed_files\jeru_metro_jtmt_border_221114.shp')

    forecast_point_subdistrict_il=forecast_point.sjoin(subdistrict_il[['geometry','ENG_NAME_nafa']])[['Taz_num','ENG_NAME_nafa']]

    forecast_point_muni_JTMT=forecast_point.query('main_sector!="Palestinian"').sjoin(muni_JTMT[['Muni_Heb', 'Sug_Muni', 'CR_PNIM', 'geometry']])[['Taz_num','Muni_Heb', 'Sug_Muni', 'CR_PNIM']]

    forecast_point_jeru_metro_jtmt_border=forecast_point.sjoin(jeru_metro_jtmt_border)[['Taz_num','jeru_metro']]

    forecast=forecast.merge(forecast_point_subdistrict_il,on='Taz_num',how='left').merge(forecast_point_muni_JTMT,on='Taz_num',how='left').merge(forecast_point_jeru_metro_jtmt_border,on='Taz_num',how='left').fillna(0)

    forecast['zonetype']=forecast['ENG_NAME_nafa']

    forecast.loc[forecast['main_sector']=='Palestinian','zonetype']='Palestinian'

    forecast['in_jerusalem_metropolin']='yes'

    forecast.loc[forecast['jeru_metro']==0,'in_jerusalem_metropolin']='no'

    forecast['yosh']=0

    forecast.loc[forecast['zonetype']=='Judea and Samaria','yosh']=1

    forecast['jerusalem_city']=1

    forecast=forecast.set_index('Taz_num')

    forecast['Taz_num']=forecast.index

    return forecast

In [4]:
forecast=add_geographical_Features(forecast)
forecast

,Name_hebre,main_sector,classification_name,Shape_Length,Shape_Area,Student_by_Classrooms,geometry,ENG_NAME_nafa,Muni_Heb,Sug_Muni,CR_PNIM,jeru_metro,zonetype,in_jerusalem_metropolin,yosh,jerusalem_city,Taz_num
Taz_num,,,,,,,,,,,,,,,,,
3008.0,חניון נצבא,Jewish,כללי צעיר,1529.406023,127367.285166,0,"MULTIPOLYGON Z (((220343.956 627834.158 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,Jerusalem,yes,0,1,3008.0
3312.0,סטריפ דרך חברון דרום,Jewish,כללי צעיר,2056.390116,129610.690438,1,"MULTIPOLYGON Z (((220257.628 627500.897 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,Jerusalem,yes,0,1,3312.0
3311.0,גבעת המטוס,Jewish,כללי צעיר,2263.979987,231733.517211,0,"MULTIPOLYGON Z (((220082.056 627554.698 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,Jerusalem,yes,0,1,3311.0
3601.0,בית צפפא מזרח,Arab,ערבי מאוזן,1489.869222,150074.543659,0,"MULTIPOLYGON Z (((220100.950 627593.769 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,Jerusalem,yes,0,1,3601.0
3009.0,סטריפ דרך חברון צפון,Jewish,כללי צעיר,1071.062705,67863.168750,1,"MULTIPOLYGON Z (((220414.404 627803.685 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,Jerusalem,yes,0,1,3009.0
3007.0,דרך חברון אשר וינר,Jewish,כללי צעיר,1560.865752,59570.861254,0,"MULTIPOLYGON Z (((220343.956 627834.158 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,Jerusalem,yes,0,1,3007.0


In [5]:
def export_forecast(forecast):
    return forecast